## Matchups Analysis - Part 1
This notebook is geared at quality assessments of the matchups runs.

In [7]:
import pandas as pd
import pickle
import glob
import os
import netCDF4 as nc

In [8]:
def ReturnL2BaseName(splitChar='_',ext='L2'):
    '''Generator yielding basename to search the matchups table'''
# Building the list of files processed
    l2fiDir = '/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/Matchups/L2/AllFiles/'
    l2Ext = '*.%s' % ext
    searchPat = os.path.join(l2fiDir,l2Ext)
    l2FiGenList = glob.iglob(searchPat)
    for l2file in l2FiGenList:
        basename = l2file.split('_')[0].split('/')[-1]
        yield basename, l2file

In [34]:
def GetTarget(df,baseName):
    return df[df['file_name'].str.contains(baseName)]

def GetUncLatLonRng(fn):
    '''Return lat/lon rng, from each file used in unc sim, as lat & lon tuples.'''
    with nc.Dataset(fn) as ds:
        naV = ds.groups['navigation_data'].variables
        latitude = naV['latitude'][:]
        longitude = naV['longitude'][:]
    latRng = (latitude.min(),latitude.max())
    lonRng = (longitude.min(),longitude.max())
    return latRng,lonRng

def CheckLocation(mUpLat,mUpLon,uncLatRng,uncLonRng):
    global summaryLatIn
    global summaryLonIn
    global totalEntries
    mLat_is_in_rng = [False] * len(mUpLon)
    mLon_is_in_rng = [False] * len(mUpLat)
    for i,e in enumerate(mUpLat):
        totalEntries += 1
        if e >= uncLatRng[0] and e <= uncLatRng[1]:
            mLat_is_in_rng[i] = True
            summaryLatIn += 1
    for i,e in enumerate(mUpLon):
        if e >= uncLonRng[0] and e <= uncLonRng[1]:
            mLon_is_in_rng[i] = True
            summaryLonIn += 1
    return mLat_is_in_rng, mLon_is_in_rng    

In [29]:
def PlotLocationComp(uncLatRng,uncLonRng,mLat,mLon):
    pass

In [10]:
# Load pandas Matchups table
dfName = 'dfMatchUpSWF.pkl'
if os.path.exists(dfName):
    dfMatchups = pd.read_pickle(dfName)
else:
    print('File not found!!')

In [36]:
baseNameGen = ReturnL2BaseName()
myUncData = ['basename','uncLatRng','uncLonRng','mUpData']
myMUpData = ['mUpNumEntries','mUpId','mUpLat','mUpLon','mUpInLatRng','mUpInLonRng']
mUpDict = dict.fromkeys(myMUpData)
mainDict = {myD:[] for myD in myUncData}
summaryLatIn,summaryLonIn,totalEntries = 0,0,0
for baseName,l2FileName in baseNameGen:
    dfSub = GetTarget(dfMatchups,baseName)
    mUpId = dfSub.index.values
    mUpLat = dfSub.latitude.values
    mUpLon = dfSub.longitude.values
    uncLatRng, uncLonRng = GetUncLatLonRng(l2FileName)
    mUpInLatRng,mUpInLonRng = CheckLocation(mUpLat,mUpLon,uncLatRng,uncLonRng)
    mainDict['basename'].append(baseName)
    mainDict['uncLatRng'].append(uncLatRng)
    mainDict['uncLonRng'].append(uncLonRng)
    mUpDict['mUpNumEntries'] = len(mUpId)
    mUpDict['mUpId'] = mUpId
    mUpDict['mUpLat'] = mUpLat
    mUpDict['mUpLon'] = mUpLon
    mUpDict['mUpInLatRng'] = mUpInLatRng
    mUpDict['mUpInLonRng'] = mUpInLonRng
    mainDict['mUpData'].append(mUpDict)
print("%d Lats and %d Lons, out of %d are in range." % (summaryLatIn, 
                                                        summaryLonIn,totalEntries))

S1999293204435
S1999046171443
S1998314160028
S1997281102712
S1997284110316
S1999226220939
S1998178131558
S1998270224215
S1998031221920
S1998101164920
S1998070200518
S1999177170758
S1998093172710
S1998190171715
S1997293192729
S1998242095953
S1998160130056
S1999043164019
S1998262200102
S1998033220958
S1999196162309
S1998104204237
S1998327172230
S1999047225905
S1998143101309
S1999188170252
S1999022205436
S1998075221010
S1997270134451
S1998197155521
S1998263204556
S1999284204006
S1998146104833
S1997331000608
S1999228202047
S1999276194102
S1998176114617
S1999271110133
S1998252023801
S1998279161421
S1998267202734
S1999255122052
S1998290224900
S1998235112130
S1998301174653
S1999187161821
S1998267170934
S1999056162710
S1998034225446
S1997270202052
S1999065212947
S1999109175018
S1999236162323
S1998287135825
S1998314173448
S1999287225334
S1998042221713
S1999041164948
S1997308172629
S1999179134007
S1999126153550
S1999341163429
S1999070070352
S1999104172550
S1999214163307
S1999187175719
S199909816

In [25]:
mUpDict

{'mUpId': array([2113]),
 'mUpInLatRng': [True],
 'mUpInLonRng': [True],
 'mUpLat': array([ 31.183]),
 'mUpLon': array([-120.935]),
 'mUpNumEntries': 1}

In [30]:
len(mainDict['basename'])

135

In [38]:
pickle.dump(mainDict,open('matchupsQADict_1.pkl','wb'))